In [ ]:
import pandas as pd
import telethon
from telethon.sync import TelegramClient
from dotenv import load_dotenv
import os
import re
import asyncio
import nest_asyncio
import csv

In [ ]:
# Apply nest_asyncio to avoid issues with event loops in interactive environments
nest_asyncio.apply()

In [ ]:
# Load environment variables from .env file
load_dotenv('.env')
api_id = os.getenv('TG_API_ID')
api_hash = os.getenv('TG_API_HASH')
phone = os.getenv('phone')

In [ ]:
# Function to scrape data from a single channel
async def scrape_channel(client, channel_username, writer, media_dir):
    entity = await client.get_entity(channel_username)
    channel_title = entity.title  # Extract the channel's title
    print(f"Scraping data from channel: {channel_title}")

    count = 0  # To count the number of messages scraped
    async for message in client.iter_messages(entity, limit=10000):
        media_path = None

        if message.media and hasattr(message.media, 'photo'):
            # Create a unique filename for the photo
            filename = f"{channel_username}_{message.id}.jpg"
            media_path = os.path.join(media_dir, filename)
            # Download the media to the specified directory if it's a photo
            await client.download_media(message.media, media_path)

        # Extract text from the message
        text = message.text if message.text else ""

        # Write message data to CSV (using writer for lists)
        writer.writerow([channel_title, message.id, message.date, text, media_path])

        count += 1

    print(f"Finished scraping {count} messages from {channel_username}.\n")